In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="239dc480-600f-413c-ae8c-8a5bbc17ed87",
    resource_group_name="rg-ml-jef",
    workspace_name="mltry",
)

In [ ]:
%pip install -U azureml-fsspec

In [2]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import os

os.environ['http_proxy'] = 'http://irproxy:8082/'
os.environ['https_proxy'] = 'http://irproxy:8082'

# update the 'my_path' variable to match the location of where you downloaded the data on your
# local filesystem

my_path = "./data/wind_bench.parquet"
# set the version number of the data asset
v1 = "initial"

my_data = Data(
    name="wind-bench",
    version=v1,
    description="Wind Benchmark parquet file",
    path=my_path,
    tags={"format": "parquet"},
    type=AssetTypes.URI_FOLDER,
)

## create data asset if it doesn't already exist:
try:
    data_asset = ml_client.data.get(name="wind-bench", version=v1)
    print(
        f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}"
    )
except:
    ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

In [3]:
print(data_asset.path)

azureml://subscriptions/239dc480-600f-413c-ae8c-8a5bbc17ed87/resourcegroups/rg-ml-jef/workspaces/mltry/datastores/workspaceblobstore/paths/LocalUpload/e371ee7f73e78305a4dda0ecbf7ff7c9/wind_bench.parquet/


In [4]:
import pandas as pd

df = pd.read_parquet(data_asset.path)

In [5]:
df.columns

Index(['Rotor azimuth [deg]', 'Rotor rotational speed [rpm]',
       'Blade 1 pitch angle [deg]', 'Blade 2 pitch angle [deg]',
       'Blade 3 pitch angle [deg]', 'Generator LSS torque [Nm]',
       'Electrical power [W]', 'Bedframe moment [Nm]',
       'Generator HSS torque [Nm]', 'Mudline moment Mx[kNm]',
       ...
       'Free wind speed Vy pos   55.00,   0.00, -95.00',
       'Free wind speed Vz pos   55.00,   0.00, -95.00',
       'Free wind speed Vx pos  -55.00,   0.00,-205.00',
       'Free wind speed Vy pos  -55.00,   0.00,-205.00',
       'Free wind speed Vz pos  -55.00,   0.00,-205.00',
       'Free wind speed Vx pos  -55.00,   0.00, -95.00',
       'Free wind speed Vy pos  -55.00,   0.00, -95.00',
       'Free wind speed Vz pos  -55.00,   0.00, -95.00',
       'Water surface elevation [m]', 'id'],
      dtype='object', length=148)

In [6]:
x_columns = [
        "Tower top fore-aft acceleration ay [m/s2]",
        "Tower top side-side acceleration ax [m/s2]",
        "Tower mid fore-aft acceleration ay [m/s2]",
        "Tower mid side-side acceleration ax [m/s2]",
        "Tower top rotation x [deg]",
        "Tower top rotation y [deg]",
        "Tower mid rotation x [deg]",
        "Tower mid rotation y [deg]"
]
    
y_columns = [
    "Mudline moment Mx[kNm]",
    "Mudline moment My[kNm]",
    "Mudline moment Mz[kNm]",
    "Waterline moment Mx[kNm]",
    "Waterline moment My[kNm]",
    "Waterline moment Mz[kNm]"
]



In [18]:
x_stats = dict(df[x_columns].describe())
for i in range(len(x_columns)):
    print(x_columns[i], x_stats[x_columns[i]]['min'], x_stats[x_columns[i]]['max'])

Tower top fore-aft acceleration ay [m/s2] -0.5544912177000001 0.59843036844
Tower top side-side acceleration ax [m/s2] -0.9779407769599999 1.0109897433600001
Tower mid fore-aft acceleration ay [m/s2] -2.0911102128 1.936147206
Tower mid side-side acceleration ax [m/s2] -1.3830194681599999 1.4393396732
Tower top rotation x [deg] -0.36000000000001364 0.6187500000000057
Tower top rotation y [deg] -0.33041424738 -0.011993925899999999
Tower mid rotation x [deg] -0.05624999999997726 0.4724999999999966
Tower mid rotation y [deg] -0.132215069246 0.001896579072


In [11]:
df[y_columns].describe()

,Mudline moment Mx[kNm],Mudline moment My[kNm],Mudline moment Mz[kNm],Waterline moment Mx[kNm],Waterline moment My[kNm],Waterline moment Mz[kNm]
count,102400.000000,102400.000000,102400.000000,102400.000000,102400.000000,102400.000000
mean,66303.954038,7338.135348,-626.049489,146700.024153,18914.916460,-626.046926
std,26975.558005,4674.049556,6683.212692,61883.337431,10821.991611,6676.654143
min,-8743.046824,-8491.565472,-38105.685644,-16593.317216,-8880.563304,-37991.438162
25%,48927.330588,4108.952077,-4608.203689,108681.237703,11469.558578,-4602.974044
50%,67752.833590,6572.670038,-359.870065,149067.356760,17287.678759,-361.536874
75%,86908.622738,9578.781101,3783.586337,194015.013249,24299.767040,3781.904644
max,136384.766550,32878.249170,29324.645220,293800.671846,73725.169680,29234.712690


In [8]:
series = {k: v for k, v in df.groupby('id')}
series['Exp1.1'].head()

,Rotor azimuth [deg],Rotor rotational speed [rpm],Blade 1 pitch angle [deg],Blade 2 pitch angle [deg],Blade 3 pitch angle [deg],Generator LSS torque [Nm],Electrical power [W],Bedframe moment [Nm],Generator HSS torque [Nm],Mudline moment Mx[kNm],...,"Free wind speed Vy pos 55.00, 0.00, -95.00","Free wind speed Vz pos 55.00, 0.00, -95.00","Free wind speed Vx pos -55.00, 0.00,-205.00","Free wind speed Vy pos -55.00, 0.00,-205.00","Free wind speed Vz pos -55.00, 0.00,-205.00","Free wind speed Vx pos -55.00, 0.00, -95.00","Free wind speed Vy pos -55.00, 0.00, -95.00","Free wind speed Vz pos -55.00, 0.00, -95.00",Water surface elevation [m],id
Time,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:51.210000,125.491519,5.069791,0.000531,0.000531,0.000531,-8.819962e+06,4520766.702,0.0,-8.819962e+06,53983.62800,...,8.995823,0.129825,-3.410884,7.417552,0.309085,-1.087036,6.050485,-0.818903,-0.190988,Exp1.1
0 days 00:00:51.260000,127.010242,5.070079,0.000531,0.000531,0.000531,-8.824104e+06,4523432.494,0.0,-8.824104e+06,53663.18725,...,8.991990,0.166382,-3.399229,7.424240,0.285645,-1.144891,6.018922,-0.837860,-0.241573,Exp1.1
0 days 00:00:51.310000,128.528965,5.068673,0.000531,0.000531,0.000531,-8.820176e+06,4520475.131,0.0,-8.820176e+06,53264.08700,...,8.991340,0.210677,-3.378393,7.431984,0.268171,-1.207049,5.985420,-0.850278,-0.288491,Exp1.1
0 days 00:00:51.360000,130.056688,5.066654,0.000531,0.000531,0.000531,-8.806109e+06,4511478.083,0.0,-8.806109e+06,52834.68350,...,8.991795,0.266327,-3.375861,7.438085,0.287174,-1.240493,5.955371,-0.884662,-0.333039,Exp1.1
0 days 00:00:51.410000,131.577661,5.064923,0.000531,0.000531,0.000531,-8.787971e+06,4500606.650,0.0,-8.787971e+06,52405.92475,...,8.993224,0.316584,-3.373300,7.444479,0.302958,-1.272780,5.924838,-0.928391,-0.373628,Exp1.1


In [ ]:
import matplotlib.pyplot as plt

serie = series['Exp1.1']
serie[['Electrical power [W]']].plot()


In [4]:
from azure.ai.ml import command

job = command(
    inputs=dict(
        data=data_asset.path,
        test_train_ratio=0.2,
        learning_rate=0.25
    ),
    code="./src/",  # location of source code
    command="python test.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}}",
    environment="torch_mlflow@latest",
    display_name="test_parquet-8",
    compute=None
)

In [5]:
ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.01 MBs): 100%|███████

Experiment,Name,Type,Status,Details Page
get-started-notebooks,nifty_yuca_7jyl5wyvqp,command,Starting,Link to Azure Machine Learning studio


In [10]:
from azureml.fsspec import AzureMachineLearningFileSystem
import pyarrow.parquet as pq

import pandas as pd

print(data_asset.path)

# instantiate file system using following URI
fs = AzureMachineLearningFileSystem(data_asset.path)

dirs = fs.glob('**/*.parquet') # list folders/files in datastore 'datastorename'

# output example:
# folder1
# folder2
# file3.csv
print(dirs[0])


#table = pq.read_pandas(dirs[0], filesystem=fs)
#print(table)
with fs.open(dirs[0]) as f:
    df = pd.read_parquet(f)
df.head()


azureml://subscriptions/239dc480-600f-413c-ae8c-8a5bbc17ed87/resourcegroups/rg-ml-jef/workspaces/mltry/datastores/workspaceblobstore/paths/LocalUpload/e371ee7f73e78305a4dda0ecbf7ff7c9/wind_bench.parquet/
LocalUpload/e371ee7f73e78305a4dda0ecbf7ff7c9/wind_bench.parquet/id=Exp0.0/2908f0244c774e8786ae9acf46715df3-0.parquet


,Rotor azimuth [deg],Rotor rotational speed [rpm],Blade 1 pitch angle [deg],Blade 2 pitch angle [deg],Blade 3 pitch angle [deg],Generator LSS torque [Nm],Electrical power [W],Bedframe moment [Nm],Generator HSS torque [Nm],Mudline moment Mx[kNm],...,"Free wind speed Vx pos 55.00, 0.00, -95.00","Free wind speed Vy pos 55.00, 0.00, -95.00","Free wind speed Vz pos 55.00, 0.00, -95.00","Free wind speed Vx pos -55.00, 0.00,-205.00","Free wind speed Vy pos -55.00, 0.00,-205.00","Free wind speed Vz pos -55.00, 0.00,-205.00","Free wind speed Vx pos -55.00, 0.00, -95.00","Free wind speed Vy pos -55.00, 0.00, -95.00","Free wind speed Vz pos -55.00, 0.00, -95.00",Water surface elevation [m]
Time,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00.010000,66.556067,7.672028,0.520026,0.519916,0.519917,-1.948289e+07,1.510994e+07,0.0,-1.948289e+07,122300.083752,...,-3.559559,8.530156,-0.126334,-2.942660,8.313082,-0.242131,-3.114716,11.832829,0.599726,-0.058321
0 days 00:00:00.060000,68.860026,7.677732,0.528227,0.528132,0.528133,-1.958159e+07,1.519899e+07,0.0,-1.958159e+07,123523.029784,...,-3.587884,8.510981,-0.104801,-2.976881,8.327318,-0.185054,-3.119303,11.840616,0.539415,-0.098596
0 days 00:00:00.110000,71.166235,7.679375,0.536139,0.536058,0.536045,-1.961654e+07,1.523091e+07,0.0,-1.961654e+07,123776.701048,...,-3.625911,8.496323,-0.142878,-2.987754,8.348031,-0.153502,-3.112289,11.845533,0.438913,-0.142723
0 days 00:00:00.160000,73.470194,7.677345,0.543834,0.543752,0.543754,-1.961114e+07,1.522401e+07,0.0,-1.961114e+07,122898.246856,...,-3.632668,8.491040,-0.183655,-3.006875,8.365335,-0.150289,-3.137754,11.833485,0.328324,-0.188766
0 days 00:00:00.210000,75.767403,7.673720,0.551354,0.551258,0.551260,-1.958625e+07,1.519774e+07,0.0,-1.958625e+07,121087.315888,...,-3.616679,8.491636,-0.221247,-3.031143,8.379997,-0.167236,-3.183398,11.810534,0.216735,-0.237811
